Use a ML algorithm to predict the wine quality
========================================

The dataset contains quality ratings (labels) for a *6497* wines samples. 

The features are the wine's physical and chemical properties (11 features). 

We want to use these properties to predict the quality of the wine (1 label)


In [1]:
DROP PROCEDURE IF EXISTS PyTrainTestSplit;
GO

CREATE PROCEDURE [dbo].[PyTrainTestSplit] (@pct int)
AS

--create the sample TRAINING table 
DROP TABLE IF EXISTS dbo.wine_sample_training
SELECT 
	* 
into wine_sample_training 
FROM WineQuality.dbo.wine_data 
WHERE (ABS(CAST(BINARY_CHECKSUM(facidity)  as int)) % 100) < @pct

--create the sample TESTING table 
DROP TABLE IF EXISTS dbo.wine_sample_testing
SELECT 
	* 
into wine_sample_testing 
FROM WineQuality.dbo.wine_data
WHERE (ABS(CAST(BINARY_CHECKSUM(facidity)  as int)) % 100) > @pct

GO

Commands completed successfully.

Total execution time: 00:00:00.0071822

Commands completed successfully.

Total execution time: 00:00:00.0062005

Split the data
========================================

In [2]:
EXEC PyTrainTestSplit 75
GO

(3775 rows affected)

(2722 rows affected)

Total execution time: 00:00:00.0463714

Create the Model
================

In [4]:
DROP PROCEDURE IF EXISTS PyTrain;
GO

CREATE PROCEDURE [dbo].[PyTrain] (
  @trained_model varbinary(max) OUTPUT
)
AS
BEGIN
EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
import numpy
import pickle
from sklearn.linear_model import LogisticRegression
from revoscalepy import RxLocalSeq, RxInSqlServer, rx_get_compute_context, rx_set_compute_context
from revoscalepy import rx_exec, rx_data_step
from microsoftml import rx_logistic_regression, rx_fast_trees
import pandas as pd

cols = InputDataSet.columns.drop(["quality", "color"])
ds = InputDataSet.drop("color",axis=1).apply(pd.to_numeric)
mod = rx_fast_trees("quality ~" + "+".join(cols), data=ds, method="regression")
trained_model = pickle.dumps(mod)
',
@input_data_1 = N'SELECT * FROM [WineQuality].[dbo].[wine_sample_training]
',
@input_data_1_name = N'InputDataSet',
@params = N'@trained_model varbinary(max) OUTPUT',
@trained_model = @trained_model OUTPUT;
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.0077219

Commands completed successfully.

Total execution time: 00:00:00.0168187

Can we find a better model?
--------------------

As is standard for creating and deploying machine learning models, it's important to try several algorithms before you deploy one, to make sure you're using a model that produces the most accurate results. Here are some other built-in packages that we could try:

- rx_btrees
- rx_fast_trees
- rx_dforest


For the purpose of this exercise, we'll just assume this is the best model after our experimentation.

Train & store the model into a sql table
====================

In [5]:
DROP TABLE IF EXISTS dbo.wine_quality_models;
GO

CREATE TABLE dbo.wine_quality_models (
    model_name varchar(255),
    model varbinary(max)
);
GO

/* Train Model */
DECLARE @model VARBINARY(MAX);
EXEC PyTrain @model OUTPUT;

/* Store the Model into a sql table */
INSERT INTO dbo.wine_quality_models (model_name, model) 
VALUES('Pytrain_model', @model);
GO

Commands completed successfully.

Total execution time: 00:00:00.0077702

Commands completed successfully.

Total execution time: 00:00:00.0069900

STDOUT message(s) from external script: 
'unbalanced_sets' ignored for method 'regression'
Not adding a normalizer.
Making per-feature arrays
Changing data from row-wise to column-wise
Beginning processing data.
Rows Read: 3775, Read Time: 0.005, Transform Time: 0
Beginning processing data.
Processed 3775 instances
Binning and forming Feature objects
Reserved memory for tree learner: 267384 bytes
Starting to train ...
Not training a calibrator because it is not needed.
Elapsed time: 00:00:02.6709488
Elapsed time: 00:00:00.3991028

(1 row affected)

Total execution time: 00:00:21.1852842

Predict the quality of the wine (using as parameters: @model and @input-data)
====================

In [6]:
DROP PROCEDURE IF EXISTS dbo.PredictQuality;
GO 

CREATE PROCEDURE [dbo].[PredictQuality] (@model varchar(50), @inquery nvarchar(max))
AS
BEGIN
DECLARE @lmodel2 varbinary(max) = (select model from dbo.wine_quality_models where model_name = @model);
EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
import pickle;
import numpy;
import pandas as pd
from sklearn import metrics
from microsoftml import rx_predict
ds = InputDataSet.drop("color",axis=1).apply(pd.to_numeric)
mod = pickle.loads(lmodel2)
X = InputDataSet[["facidity", "vacidity", "citric", "sugar", "chlorides", "fsulfur", "tsulfur", "density", "pH", "sulphates", "alcohol"]]
#y = numpy.ravel(InputDataSet[["quality"]])
pred_list = rx_predict(model = mod, data = X, extra_vars_to_write=["quality_Pred"])

#OutputDataSet = pd.DataFrame({"quality": InputDataSet[["quality"]], "quality_Pred": round(pred_list,1)})
OutputDataSet = round(pred_list,1)
',	
  @input_data_1 = @inquery,
  @input_data_1_name = N'InputDataSet',
  @params = N'@lmodel2 varbinary(max)',
  @lmodel2 = @lmodel2
WITH RESULT SETS ((Score float));
END;

Commands completed successfully.

Total execution time: 00:00:00.0152787

Commands completed successfully.

Total execution time: 00:00:00.0077578

Execute Stored Procedure on unseen data (testing data set)
====================

In [7]:
DECLARE @query_string nvarchar(max)
  SET @query_string='
  select * from dbo.wine_sample_testing'
EXEC [dbo].[PredictQuality] 'Pytrain_model', @query_string;

STDOUT message(s) from external script: 
Beginning processing data.
Rows Read: 2722, Read Time: 0.001, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:01.5796885
Finished writing 2722 rows.
Writing completed.

(2722 rows affected)

Total execution time: 00:00:08.0770831

Score
5.6
5.7
5.7
5.6
6
5.5
4.7
5.6
5.9
6.1
